In [23]:
import glob
import numpy as np

from lib import training as tr

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [24]:
batch_size = 128
epochs = 200

In [25]:
x_train, y_train = tr.prepare_data('csv/loto_x.csv', 'csv/loto_y.csv')
#x_train = tr.fit_transform(x_train)
x_train = x_train.astype('float32')
x_train /= tr.num_classes
y_train -= 1
y_train = keras.utils.to_categorical(y_train, tr.num_classes)

In [26]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(len(x_train[0]),)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(tr.num_classes, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               16896     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 43)                22059     
Total params: 301,611
Trainable params: 301,611
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1)

Epoch 1/200
10514/10514 [==============================] - 7s 710us/step - loss: 3.7708 - accuracy: 0.0227
Epoch 2/200
10514/10514 [==============================] - 7s 670us/step - loss: 3.7636 - accuracy: 0.0241
Epoch 3/200
10514/10514 [==============================] - 7s 667us/step - loss: 3.7617 - accuracy: 0.0238
Epoch 4/200
10514/10514 [==============================] - 7s 676us/step - loss: 3.7600 - accuracy: 0.0255
Epoch 5/200
10514/10514 [==============================] - 7s 682us/step - loss: 3.7585 - accuracy: 0.0261
Epoch 6/200
10514/10514 [==============================] - 7s 684us/step - loss: 3.7565 - accuracy: 0.0279
Epoch 7/200
10514/10514 [==============================] - 7s 671us/step - loss: 3.7544 - accuracy: 0.0296
Epoch 8/200
10514/10514 [==============================] - 7s 673us/step - loss: 3.7529 - accuracy: 0.0315
Epoch 9/200
10514/10514 [==============================] - 7s 692us/step - loss: 3.7509 - accuracy: 0.0322
Epoch 10/200
10514/10514 [===========

In [29]:
model.save(tr.h5_file, include_optimizer = True)